In [1]:
from lifelines import CoxPHFitter
import pandas as pd
from lifelines.utils import k_fold_cross_validation
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA

In [31]:
import numpy as np

In [ ]:
# mrna-meth 454
# mrna-mirna 447
# mrna-meth-mirna 447

In [27]:
clinical_mrna = pd.read_csv('clinical_data_subset/clinical_subset_mrna.csv')
clinical_meth = pd.read_csv('clinical_data_subset/clinical_subset_meth.csv')
clinical_mirna = pd.read_csv('clinical_data_subset/clinical_subset_mirna.csv')
clinical = pd.read_csv('clinical_data_subset/clinical_data.csv')

In [61]:
mrna = pd.read_csv('data/clean_datasets/mrna_clean.csv')
meth = pd.read_csv('data/clean_datasets/meth_clean.csv')
mirna = pd.read_csv('data/clean_datasets/mirna_clean.csv')

In [4]:
meth  = meth.drop(columns=['Unnamed: 0'])

In [5]:
def make_df(df):
    df = df.T
    headers = df.iloc[0]
    return pd.DataFrame(df.values[1:], columns=headers)

In [62]:
df_mrna = make_df(mrna)
df_meth = make_df(meth)
df_mirna = make_df(mirna)

In [83]:
# headers = df_meth.iloc[0]
# df_meth = pd.DataFrame(df_meth.values[1:], columns=headers)
df_mrna.shape, df_meth.shape, df_mirna.shape

((515, 20172), (458, 17052), (450, 477))

In [77]:
#df_mrna = df_mrna.reset_index(drop=True)
#df_mrna = df_mrna.set_index([clinical_mrna['sample'].values])
#df_meth = df_meth.reset_index(drop=True)
#df_meth = df_meth.set_index([clinical_meth['sample'].values])


In [82]:
df_mirna = df_mirna.reset_index(drop=True)
df_mirna = df_mirna.set_index([clinical_mirna['sample'].values])

In [84]:
clinical_mrna.shape,clinical_meth.shape,clinical_mirna.shape

((515, 10), (458, 10), (450, 10))

In [125]:
mrna_meth = list(set(clinical_mrna['sample'].tolist()) & set(clinical_meth['sample'].tolist()))
mrna_meth.sort()

In [126]:
mrna_mirna = list(set(clinical_mrna['sample'].tolist()) & set(clinical_mirna['sample'].tolist()))
mrna_mirna.sort()

In [127]:
mrna_meth_mirna =list(set(clinical_mrna['sample'].tolist()) & set(clinical_meth['sample'].tolist()) & set(clinical_mirna['sample'].tolist()))
mrna_meth_mirna.sort()

In [209]:
# these sets are equal
mrna_mirna == mrna_meth_mirna

True

In [128]:
clinical_mrna_meth = clinical.loc[clinical['sample'].isin(mrna_meth)]

In [183]:
clinical_mrna_mirna = clinical.loc[clinical['sample'].isin(mrna_mirna)]

In [129]:
clinical_mrna_meth = clinical_mrna_meth.reset_index(drop=True)
clinical_mrna_meth = clinical_mrna_meth.set_index([mrna_meth])

In [185]:
clinical_mrna_mirna = clinical_mrna_mirna.reset_index(drop=True)
clinical_mrna_mirna = clinical_mrna_mirna.set_index([mrna_mirna])

In [141]:
df_mrna['sample'] = clinical_mrna['sample'].values


In [187]:
#df_mrna_common = df_mrna[df_mrna['sample'].isin(mrna_meth)]
#df_mrna_common = df_mrna_common.reset_index(drop=True)
#df_mrna_common = df_mrna_common.set_index([mrna_meth])

df_mrna_common = df_mrna[df_mrna['sample'].isin(mrna_mirna)]
df_mrna_common = df_mrna_common.reset_index(drop=True)
df_mrna_common = df_mrna_common.set_index([mrna_mirna])

In [189]:
df_mrna_common = df_mrna_common.drop(columns=['sample'])
#mrna common between mrna and meth completed

In [210]:
df_meth['sample'] = clinical_meth['sample'].values

In [211]:
df_meth_common = df_meth[df_meth['sample'].isin(mrna_mirna)]

In [212]:
df_meth_common = df_meth_common.reset_index(drop=True)
df_meth_common = df_meth_common.set_index([mrna_mirna])

In [213]:
df_meth_common = df_meth_common.drop(columns=['sample'])
#meth common between mrna and meth completed

In [190]:
#starting for mirna
df_mirna['sample'] = clinical_mirna['sample'].values
df_mirna_common = df_mirna[df_mirna['sample'].isin(mrna_mirna)]
df_mirna_common = df_mirna_common.reset_index(drop=True)
df_mirna_common = df_mirna_common.set_index([mrna_mirna])
df_mirna_common = df_mirna_common.drop(columns=['sample'])
#mirna completed

In [214]:
df_meth_common.shape

(447, 17052)

In [193]:
df_mrna_common.shape, df_mirna_common.shape

((447, 20172), (447, 477))

In [215]:
# Make an instance of the Model
#mrna_pca = PCA(.95)
meth_pca = PCA(.95)
#mirna_pca = PCA(.95)

In [216]:
#mrna_pca.fit(df_mrna_common)
meth_pca.fit(df_meth_common)
#mirna_pca.fit(df_mirna_common)

PCA(copy=True, iterated_power='auto', n_components=0.95, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [217]:
print(sum(mrna_pca.explained_variance_ratio_), mrna_pca.n_components_)
print(sum(meth_pca.explained_variance_ratio_), meth_pca.n_components_)
print(sum(mirna_pca.explained_variance_ratio_), mirna_pca.n_components_)

0.95003798812105 333
0.9502741398598527 308
0.9500370517753948 163


In [198]:
mrna_dataset = pd.DataFrame(data = mrna_pca.transform(df_mrna_common), columns = [ 'mrna' + str(i) for i in range(333) ])

In [218]:
meth_dataset = pd.DataFrame(data = meth_pca.transform(df_meth_common),columns = [ 'meth' + str(i) for i in range(308)])

In [199]:
mirna_dataset = pd.DataFrame(data = mirna_pca.transform(df_mirna_common),columns = [ 'mirna' + str(i) for i in range(163)])

In [219]:
mrna_dataset.shape, meth_dataset.shape, mirna_dataset.shape

((447, 333), (447, 308), (447, 163))

In [220]:
dataset = pd.concat([mrna_dataset, meth_dataset, mirna_dataset], axis=1)

In [221]:
#dataset['time'] = clinical_mrna_meth['os_time'].values
#dataset['event'] = clinical_mrna_meth['vital_status'].values
#below mrna-mirna
dataset['time'] = clinical_mrna_mirna['os_time'].values
dataset['event'] = clinical_mrna_mirna['vital_status'].values


In [222]:
dataset

,mrna0,mrna1,mrna2,mrna3,mrna4,mrna5,mrna6,mrna7,mrna8,mrna9,...,mirna155,mirna156,mirna157,mirna158,mirna159,mirna160,mirna161,mirna162,time,event
0,-28.924205,-2.256348,-41.736008,14.621405,-3.370295,-12.048357,2.896336,-9.505792,24.869736,-5.902621,...,0.441129,0.187670,-0.202608,0.164466,-0.451361,0.218860,-0.508364,-0.761670,426,0
1,54.255508,-5.609267,-5.090153,3.631049,6.099151,-21.333176,-64.921481,0.919276,-28.909864,4.431661,...,-1.043746,0.071673,1.819382,0.666579,-0.103264,0.185177,-0.550752,0.615946,1126,0
2,7.230536,19.766549,-66.919828,16.357224,-11.326349,-30.311429,19.328881,-9.466862,-13.577853,-17.152247,...,-0.711154,-1.121161,-0.976051,1.421195,-0.592723,0.700416,-1.155163,-0.522876,303,1
3,-28.052783,13.780466,-5.130901,-5.944914,-27.262480,18.320223,-20.172120,14.862320,2.244646,-19.303214,...,0.534318,0.144796,-0.766692,0.461012,0.768809,-0.758964,-0.156005,0.900483,610,0
4,-18.985417,23.822837,-12.369518,0.514395,-23.290519,21.607868,5.853372,-8.081999,-12.579667,3.260301,...,-0.592657,0.018165,1.393010,-0.392463,0.093771,-0.247321,0.927634,0.202288,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
442,-22.795448,-25.656823,-15.244821,8.496399,-13.564654,-32.383883,12.524326,-16.224547,2.017403,6.412014,...,0.113588,-1.057692,0.755743,-0.677567,0.283996,0.050445,-0.251194,-0.390475,13,0
443,-5.593109,-30.178377,-28.200395,-54.724530,45.556637,-34.295215,-2.169417,10.924225,2.559223,-10.711207,...,0.514252,-0.681425,-0.609325,-1.426887,0.020932,0.755544,-0.389962,0.098624,603,0
444,-8.912616,-67.056979,-43.284510,14.512657,-27.344568,12.536661,8.025054,-4.834176,-9.832838,12.039606,...,0.629902,-0.715716,0.265743,-0.169176,-0.542917,0.150253,-0.229927,-0.189729,617,0
445,-44.764026,-26.879767,-18.214252,-5.431014,-15.595680,25.244400,-0.130274,-4.586285,-10.217744,15.316511,...,0.966239,0.201114,-0.979356,-0.137781,-0.243150,0.930070,-0.424821,-0.212150,1798,1


In [16]:
dataset = dataset.reset_index(drop=True)
dataset = dataset.set_index([clinical_meth['sample'].values])

In [17]:
def avg(in_list):
    return sum(in_list)/len(in_list)

In [245]:
del clinical_feat
del clinical_data

In [254]:
#onehot clinical
enc = OneHotEncoder(handle_unknown='ignore')
clinical_data = clinical_mrna_mirna
one_hot_T = pd.DataFrame(enc.fit_transform(clinical_data[['pathology_T_stage']]).toarray(), columns=['tf1','tf2','tf3','tf4','tf5','tf6','tf7','tf8','tf9'])
one_hot_N = pd.DataFrame(enc.fit_transform(clinical_data[['pathology_N_stage']]).toarray(), columns=['nf1','nf2','nf3','nf4','nf5'])
one_hot_M = pd.DataFrame(enc.fit_transform(clinical_data[['pathology_M_stage']]).toarray(),
columns=['mf1','mf2','mf3','mf4','mf5'])
one_hot_G = pd.DataFrame(enc.fit_transform(clinical_data[['gender']]).toarray(),columns=['gf1','gf2'])

clinical_feat =pd.concat([one_hot_T,one_hot_N, one_hot_M, one_hot_G], axis=1)
clinical_feat = clinical_feat.reset_index(drop=True)
#clinical_feat = clinical_feat.set_index([clinical_data['sample'].values])

#train_id = []
#for patient in tqdm(training_list):
#    train_id.append(patient.split('.')[0])
#clinical_feat = clinical_feat.loc[train_id,:]
#print("Features Processed")

In [259]:
clinical_feat['age'] = clinical_data['age'].values

In [260]:
clinical_feat

,tf1,tf2,tf3,tf4,tf5,tf6,tf7,tf8,tf9,nf1,...,nf4,nf5,mf1,mf2,mf3,mf4,mf5,gf1,gf2,age
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,66
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,58
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,76
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,74
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
442,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,56
443,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,67
444,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,49
445,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,74


In [261]:
omic_clin = pd.concat([dataset, clinical_feat], axis=1)

In [262]:
omic_clin

,mrna0,mrna1,mrna2,mrna3,mrna4,mrna5,mrna6,mrna7,mrna8,mrna9,...,nf4,nf5,mf1,mf2,mf3,mf4,mf5,gf1,gf2,age
0,-28.924205,-2.256348,-41.736008,14.621405,-3.370295,-12.048357,2.896336,-9.505792,24.869736,-5.902621,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,66
1,54.255508,-5.609267,-5.090153,3.631049,6.099151,-21.333176,-64.921481,0.919276,-28.909864,4.431661,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,58
2,7.230536,19.766549,-66.919828,16.357224,-11.326349,-30.311429,19.328881,-9.466862,-13.577853,-17.152247,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,76
3,-28.052783,13.780466,-5.130901,-5.944914,-27.262480,18.320223,-20.172120,14.862320,2.244646,-19.303214,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,74
4,-18.985417,23.822837,-12.369518,0.514395,-23.290519,21.607868,5.853372,-8.081999,-12.579667,3.260301,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
442,-22.795448,-25.656823,-15.244821,8.496399,-13.564654,-32.383883,12.524326,-16.224547,2.017403,6.412014,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,56
443,-5.593109,-30.178377,-28.200395,-54.724530,45.556637,-34.295215,-2.169417,10.924225,2.559223,-10.711207,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,67
444,-8.912616,-67.056979,-43.284510,14.512657,-27.344568,12.536661,8.025054,-4.834176,-9.832838,12.039606,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,49
445,-44.764026,-26.879767,-18.214252,-5.431014,-15.595680,25.244400,-0.130274,-4.586285,-10.217744,15.316511,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,74


In [32]:
#omic_clin_corr  = omic_clin.corr().abs()

In [33]:
#upper = omic_clin_corr.where(np.triu(np.ones(omic_clin_corr.shape),k=1).astype(np.bool))

In [54]:
#to_drop = [column for column in upper.columns if any(upper[column] > 0.5)]

In [56]:
#omic_clin = omic_clin.drop(columns=['nf2', 'nf3', 'nf5'])

In [263]:
#cox model
cph = CoxPHFitter(penalizer=0.1)
#cph.fit(dataset, duration_col='time', event_col='event', show_progress=True)
#scores = k_fold_cross_validation(cph, dataset, duration_col='time', event_col='event', k=10, scoring_method="concordance_index")
scores = k_fold_cross_validation(cph, omic_clin, duration_col='time', event_col='event', k=10, scoring_method="concordance_index")

ConvergenceError: Convergence halted due to matrix inversion problems. Suspicion is high collinearity. Please see the following tips in the lifelines documentation: https://lifelines.readthedocs.io/en/latest/Examples.html#problems-with-convergence-in-the-cox-proportional-hazard-modelMatrix is singular.

In [264]:
scores

[0.5214723926380368,
 0.5689655172413793,
 0.6311239193083573,
 0.5552325581395349,
 0.6047904191616766,
 0.4805491990846682,
 0.6157407407407407,
 0.5128805620608899,
 0.6666666666666666,
 0.5657142857142857]

In [25]:
avg(scores)

0.5644322828561628

In [ ]:
master_clinical = pd.read_csv('clinical_data_subset/clinical_data.csv')

In [ ]:
master_clinical.shape